In [1]:
import pandas as pd
import numpy as np

In [2]:
# Upload repairs data
dtp = pd.read_excel('C:/Users/sgulbin/Work/_Выгрузки_и_Расчеты/accidents_4_10_2020.xlsx')
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='4/18/2020 12:00:00 AM', '3/18/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='4/20/2020 12:00:00 AM', '3/20/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='5/24/2020 12:00:00 AM', '3/24/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='7/23/2020 12:00:00 AM', '3/23/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = pd.to_datetime(dtp['Дата и время ДТП'], dayfirst=False, errors = 'coerce')
dtp.sort_values('Дата и время ДТП', inplace=True)
dtp['Дата и время ДТП'] = pd.to_datetime(dtp['Дата и время ДТП'])
dtp = dtp[dtp['Телефон водителя'].notnull()]
dtp['Телефон водителя'] = dtp['Телефон водителя'].apply(int)
dtp = dtp[dtp['ТС'].notnull()]
dtp.reset_index(drop=True, inplace = True)
dtp['ТС'] = dtp['ТС'].str.replace(' ', '')
dtp = dtp[['Дата и время ДТП', 'Регион','Телефон водителя', 'Виновность', 'ТС', 'Модель автомобиля', 'POOL', 'Тариф','Ремонт не производился','Тип ДТП', 'Вид страхования', 'Пострадавшие','Сумма по заказ-наряду','Выставлено', 'Страховая компания сумма выплаченная']].rename(columns={
    'Дата и время ДТП': 'dt',
    'Регион': 'region',
    'Телефон водителя': 'phone',
    'Виновность': 'Responsibility',
    'ТС': 'license_plate',
    'Модель автомобиля': 'vehicle_model',
    'POOL': 'pool',
    'Тариф': 'tariff',
    'Ремонт не производился': 'no repair conducted',
    'тип ДТП': 'accident type',
    'Вид страхования': 'insurance type',
    'Пострадавшие': 'injured',
    'Сумма по заказ-наряду': 'target_amount',
    'Выставлено': 'Vystavleno',
    'Страховая компания сумма выплаченная': 'insurance paid sum'
})

In [3]:
fleet = pd.read_csv('C:/Users/sgulbin/Work/Data/fleet_data.csv')
# Leaving only rentable and active cars; removing unnecessary fields
fleet = fleet.loc[(fleet.is_pool == False)&(fleet.is_active == True)&(fleet.is_rentable == True)]
fleet = fleet[['license_plate_number', 'VIN', 'region_name_en']]
# Replace latin characters with corresponding cyrillic
fleet.license_plate_number = fleet.license_plate_number.str.replace('a','А')
fleet.license_plate_number = fleet.license_plate_number.str.replace('b','В')
fleet.license_plate_number = fleet.license_plate_number.str.replace('c','С')  
fleet.license_plate_number = fleet.license_plate_number.str.replace('e','Е')
fleet.license_plate_number = fleet.license_plate_number.str.replace('h','Н')
fleet.license_plate_number = fleet.license_plate_number.str.replace('k','К') 
fleet.license_plate_number = fleet.license_plate_number.str.replace('m','М')
fleet.license_plate_number = fleet.license_plate_number.str.replace('o','О')
fleet.license_plate_number = fleet.license_plate_number.str.replace('p','Р')  
fleet.license_plate_number = fleet.license_plate_number.str.replace('t','Т')
fleet.license_plate_number = fleet.license_plate_number.str.replace('x','Х')
fleet.license_plate_number = fleet.license_plate_number.str.replace('y','У')
# Upper case VIN and license plate number for successfull merge with other df
fleet.VIN = fleet['VIN'].str.upper()
fleet.license_plate_number = fleet.license_plate_number.str.upper()

In [4]:
# Import file with statistics on cars that were moved to regions
to_regions = pd.read_csv('C:/Users/sgulbin/Work/Data/moved_to_regions.csv')
to_regions = to_regions[['Гос знак', 'VIN', 'Марка', 'Город']]
fleet = pd.merge(fleet, to_regions, on = 'VIN', how = 'left')
# Returning regions as of Q1 2020
fleet.loc[pd.isnull(fleet['Город']) == False,'fleet_region'] = 'Moscow'
fleet.loc[pd.isnull(fleet['Город']) == True,'fleet_region'] = fleet['region_name_en']
fleet = fleet[['license_plate_number', 'VIN', 'fleet_region']]

In [5]:
dtp20 = dtp.loc[(dtp.dt >= '2019-04-01 00:00:00')&(dtp.dt < '2020-04-01 00:00:00')&(dtp.pool == 'No')&\
                (dtp.Responsibility!='Не виновен')&(dtp.Responsibility != 'Фейк')&(dtp.Responsibility != 'Угон')]
dtp20 = pd.merge(dtp20,fleet,left_on = 'license_plate', right_on = 'license_plate_number', how = 'left')
# Replacing vehicle model names with correct ones
dtp20=dtp20.replace({'vehicle_model' : {'320i':'BMW 3-series', 'Kaptur':'Renault Kaptur', 'Polo': 'Volkswagen Polo', \
                                  'QASHQAI':'Nissan Qashqai','Rio':'Kia Rio', 'Solaris':'Hyundai Solaris', '3D': 'Mini Cooper',\
                                 '500': 'Fiat 500', '5D':'Mini Cooper', 'A3':'Audi A3', 'E 200': 'Mercedes-Benz E200',\
                                 'FORFOUR': 'Smart Forfour', 'FORTWO COUPE':'Smart Fortwo', 'GLC 250': 'Mercedes-Benz GLC',\
                                       'Rio X-Line': 'KIA Rio X-Line', 'Sandero':'Renault Sandero','Sportage':'Kia Sportage'}})

In [39]:
rides = pd.read_csv('C:/Users/sgulbin/Work/Data/ride_time_by_VIN_2019Q2 - 2020Q1.csv')
rides.VIN = rides['VIN'].str.upper()
ins = pd.read_excel('\\\\delimobil.local/Delimobil/Risk/Страхование/Отчет по страхованию/test/AllPolicies_Agg_2019Q2 - 2020Q1.xlsx', \
                    sheet_name='OSAGO_Prem_2019')
ins = ins[['Экспозиция', 'Заработанная премия', 'Марка/Модель ТС по ПТС', 'VIN                 ', 'ГРЗ']]
ins.columns = ['exp', 'ep', 'model', 'VIN', 'license_plates']
ins.VIN = ins['VIN'].str.upper()
ins = ins.groupby('VIN').sum()
df = pd.merge(pd.merge(fleet, ins, on = 'VIN', how = 'left'),rides, on = 'VIN', how = 'left')
by_region = df.groupby('fleet_region').sum()
by_region.to_csv('\\\\delimobil.local/Delimobil/Risk/Страхование/Убыточность 2020/exposure_by_region_2019Q2-2020Q1.csv')
print(by_region.ride_time.sum())
by_region

787549776.5300022


,exp,ep,ride_time,total_time
fleet_region,,,,
Grozny,4.436813,3.435070e+04,2.481346e+06,4.363807e+06
Krasnodar,131.939560,1.402139e+06,1.113834e+07,1.875131e+07
Krasnoyarsk,31.642857,3.095728e+05,3.666448e+06,6.801393e+06
Moscow,7541.178571,7.330189e+07,6.359808e+08,9.816213e+08
Nizhny Novgorod,87.472527,7.896758e+05,7.967846e+06,1.377584e+07
Novosibirsk,44.000000,3.832726e+05,6.007617e+06,1.046056e+07
Rostov-on-Don,36.000000,3.639089e+05,3.686280e+06,6.412083e+06
Samara,32.000000,3.069648e+05,6.696952e+06,1.141665e+07
St. Petersburg,780.315934,8.052897e+06,7.063798e+07,1.126554e+08


In [9]:
count = dtp20.groupby('region').count()
count.to_csv('\\\\delimobil.local/Delimobil/Risk/Страхование/Убыточность 2020/accidents_by_region_2019Q2-2020Q1.csv')
count

,dt,phone,Responsibility,license_plate,vehicle_model,pool,tariff,no repair conducted,Тип ДТП,insurance type,injured,target_amount,Vystavleno,insurance paid sum,license_plate_number,VIN,fleet_region
region,,,,,,,,,,,,,,,,,
Грозный,19,19,19,19,19,19,19,19,19,1,18,19,19,0,19,19,19
Екатеринбург,354,354,354,354,354,354,354,354,354,158,352,317,301,2,324,324,324
Краснодар,203,203,200,203,203,203,203,203,199,45,199,180,180,0,200,200,200
Красноярск,98,98,98,98,98,98,98,98,96,40,96,90,75,0,98,98,98
Москва,7683,7683,7621,7683,7683,7683,7512,7683,5409,2748,5388,5686,5161,28,7631,7631,7631
Нижний Новгород,213,213,213,213,213,213,213,213,211,22,208,154,189,0,183,183,183
Новосибирск,114,114,111,114,114,114,114,114,111,55,111,99,90,1,99,99,99
Ростов-на-Дону,76,76,76,76,76,76,76,76,74,33,71,68,67,0,72,72,72
Самара,114,114,114,114,114,114,114,114,111,87,109,91,87,0,110,110,110


In [38]:
rides = pd.read_csv('C:/Users/sgulbin/Work/Data/ride_time_by_VIN_2019Q2 - 2020Q1.csv')
rides.VIN = rides['VIN'].str.upper()
ins = pd.read_excel('\\\\delimobil.local/Delimobil/Risk/Страхование/Отчет по страхованию/test/AllPolicies_Agg_2019Q2 - 2020Q1.xlsx', \
                    sheet_name='OSAGO_Prem_2019')
ins = ins[['Экспозиция', 'Заработанная премия', 'Марка/Модель ТС по ПТС', 'VIN                 ', 'ГРЗ']]
ins.columns = ['exp', 'ep', 'model', 'VIN', 'license_plates']
ins.VIN = ins['VIN'].str.upper()
ins = ins.groupby('VIN').sum()
df = pd.merge(pd.merge(fleet, ins, on = 'VIN', how = 'left'),rides, on = 'VIN', how = 'left')
by_region = df.groupby('fleet_region').sum()
by_region

,exp,ep,ride_time,total_time
fleet_region,,,,
Grozny,4.436813,3.435070e+04,2.481346e+06,4.363807e+06
Krasnodar,131.939560,1.402139e+06,1.113834e+07,1.875131e+07
Krasnoyarsk,31.642857,3.095728e+05,3.666448e+06,6.801393e+06
Moscow,7541.178571,7.330189e+07,6.359808e+08,9.816213e+08
Nizhny Novgorod,87.472527,7.896758e+05,7.967846e+06,1.377584e+07
Novosibirsk,44.000000,3.832726e+05,6.007617e+06,1.046056e+07
Rostov-on-Don,36.000000,3.639089e+05,3.686280e+06,6.412083e+06
Samara,32.000000,3.069648e+05,6.696952e+06,1.141665e+07
St. Petersburg,780.315934,8.052897e+06,7.063798e+07,1.126554e+08


In [37]:
# len(fleet.VIN.unique())
# ins.loc[pd.isnull(ins.fleet_region) == True]
# test = pd.merge(ins, fleet, on = 'VIN', how = 'left')
# test.loc[pd.isnull(test.fleet_region) == True]
# check.ride_time.sum()
test = ins.groupby('VIN').sum()
test = test.loc[test.exp > 0]
test

,exp,ep
VIN,,
WAUZZZF11KD017917,0.016484,191.489011
WAUZZZF24KN032754,0.016484,191.489011
WAUZZZF58JA096960,0.079670,925.530220
WAUZZZF81JN002372,0.079670,925.530220
WBA8A51000AE75413,1.453297,18311.898132
...,...,...
ZFA312000LJC39937,0.598901,4880.852308
ZFA312000LJC40375,0.598901,4880.852308
ZFA312000LJC40383,0.598901,4880.852308
